In [1]:
import numpy as np
from AbstractMesh import AbstractMesh
import utils
from metrics import aspect_ratio
from scipy.sparse import *

In [12]:
class Trimesh(AbstractMesh):
    
    def __init__(self, vertices= None, faces = None, face_normals = None, labels = None):
        
        self.face_normals         = None #npArray (Nx3)
        self.face_labels       = None #npArray (Nx1)
        self.faces            = None #npArray (Nx3)
        self.face2face         = None #npArray (Nx3?)
        
        super(Trimesh, self).__init__()
        
        if vertices and faces:
            
            self.vertices = np.array(vertices) 
            self.faces = np.array(faces)
            
            if face_normals:
                self.face_normals = np.array(face_normals)
            
            if labels:
                self.labels = np.array(labels)
            
            self.__load_operations()
         
    
    def show(self):
        pass
    
    
    @property
    def num_faces(self):
        
        return self.faces.shape[0]

#_______________________________Operations on Mesh Elements__________________________________________________________________________________

    def add_face(self,face_id0, face_id1, face_id2):
        
        self.add_faces([face_id0, face_id1, face_id2])
        
        
    def add_faces(self, new_faces):
            
        new_faces = np.array(new_faces)
        self.faces = np.concatenate([self.faces, new_faces])
        
        if new_faces[(new_faces[:,0] > self.num_vertices) | 
                     (new_faces[:,1] > self.num_vertices) | 
                     (new_faces[:,2] > self.num_vertices)].shape[0] > self.num_vertices:
            raise Exception('The Id of a vertex must be lesser than the number of vertices')

        self.faces = np.concatenate([self.faces, new_faces])
        self.__compute_adjacencies()
        
    
    def remove_face(self,face_id):
        
        self.remove_faces([face_id])
        
        
    def remove_faces(self, face_ids):
        
        face_ids = np.array(face_ids)
        mask = np.ones(self.num_faces)
        mask[face_ids] = 0
        mask = mask.astype(np.bool)
        
        self.faces = self.faces[mask]
        self.__compute_adjacencies()
        
    
    def remove_vertex(self,vtx_id):
        
        self.remove_vertices([vtx_id])
    
    
    def remove_vertices(self, vtx_ids):
        
        for vtx_id in vtx_ids:
            
            self.vertices = np.delete(self.vertices, vtx_id)
            self.faces = self.faces[(self.faces[:,0] != vtx_id) & 
                                    (self.faces[:,1] != vtx_id) & 
                                    (self.faces[:,2] != vtx_id)]
            
            self.faces[(self.faces[:,0] > vtx_id)][0] -=1
            self.faces[(self.faces[:,1] > vtx_id)][1] -=1
            self.faces[(self.faces[:,2] > vtx_id)][2] -=1
        
        self.__compute_adjacencies()
    
#____________________________________________________________________________________________________________________________________________________
    
    def __compute_adjacencies(self):
        map_ = dict()
        adjs =  [[] for i in range(self.num_faces)]

        edges = np.c_[self.faces[:,0], self.faces[:,1], self.faces[:,1], self.faces[:,2], self.faces[:,2], self.faces[:,0]]
        edges.shape = (-1, 2)
        faces_idx = np.repeat(np.array(range(self.num_faces)), 3)


        for e, f in zip(edges, faces_idx):
            e = (e[0], e[1])

            try:
                tmp = map_[e]
            except KeyError:
                tmp = None

            if tmp is None:
                map_[(e[1], e[0])] = f
            else:
                adjs[f].append(map_[e])
                adjs[map_[e]].append(f)


        self.face2face = np.array([a.append(-1) if len(a) < 3 else a for a in adjs])

        
    def __load_operations(self):
        
        self.__compute_adjacencies()
        #self._AbstractMesh__compute_vtx2vtx()
        self._AbstractMesh__update_bounding_box()
        #self.__compute_metrics()
        
    def load_from_file(self, filename):
        
        self.vertices, self.faces, self.face_normals = utils.read_obj(filename)
        self.__load_operations()
        
    
    def save_file(self, filename):
        raise NotImplementedError
        
        
    @property
    def simplex_centroids(self):
        
        if self._AbstractMesh__simplex_centroids is None:
            self._AbstractMesh__simplex_centroids = self.vertices[self.faces].mean(axis = 1)
        
        return self._AbstractMesh__simplex_centroids
        
        
    
    def __compute_metrics(self): 
        
        self.simplex_metrics['aspect_ratio'] = aspect_ratio(self.vertices, self.faces)
        

In [9]:
vtx, faces, face_norm = utils.read_obj('bunny.obj')

In [13]:
prova = Trimesh()

In [17]:
prova.load_from_file('bunny.obj')
prova.faces

array([[ 6421,  6423, 13348],
       [13043,   200, 11256],
       [ 2903,  7731,  5401],
       ...,
       [ 9656,   976,  9626],
       [ 7219,  2710,  2709],
       [ 3209, 11163, 11748]])

In [15]:
prova.get_metric('aspect_ratio', 5)

TypeError: 'NoneType' object is not subscriptable

In [60]:
prova.num_faces

8996

In [200]:
faces[(faces[:,0] > 100) | (faces[:,1] > 100) | (faces[:,2] > 100)].shape

(28556, 3)

In [66]:
vtx[faces][:, :, 0] - vtx[faces][:, :, 1]

AttributeError: module 'numpy' has no attribute 'norm'